<a href="https://colab.research.google.com/github/quotation3/DB_Finance/blob/main/Sentiment_Lexicon/7_%EA%B0%90%EC%84%B1%EC%82%AC%EC%A0%84_%EA%B5%AC%EC%B6%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ngram 전처리

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from tqdm import tqdm

import numpy as np
import pandas as pd
import csv

In [ ]:
news = pd.read_csv('/content/drive/MyDrive/DB_Finance/data/ngram_news_only_topic.csv',lineterminator='\n')
news

,date,ngram,label
0,2017-01-02,"['한국', '배터리', '보조금', '제외', '화학', '삼성', '날벼락', ...",0
1,2017-01-02,"['리더십', '시대', '분석', '산업', '혁명', '리더', '경영인', '...",0
2,2017-01-02,"['구본무', '회장', '창립', '올해', '시작', '구본무', '회장', '...",0
3,2017-01-02,"['한국', '배터리', '설', '자리', '장착', '차량', '보조금', '제...",0
4,2017-01-02,"['충전', '시간', '사용', '노트북', '시판', '충전', '시간', '사...",0
...,...,...,...
2878,2019-12-30,"['화학', '놀이터', '화학', '캠프', '과학', '교육', '기부', '화...",1
2879,2019-12-30,"['배터리', '시가총액', '돌파', '배터리', '시가총액', '돌파', '세계...",1
2880,2019-12-30,"['전자', '폴란드', '브로츠와프', '배터리', '분리', '공장', '증설'...",1
2881,2019-12-30,"['결산', '산업', '소송', '화재', '다사다난', '배터리', '업계', ...",1


In [ ]:
for i in range(len(news)):
    try:
        news['ngram'][i] = ''.join(news['ngram'][i].replace('[','').replace(']','').replace("'",''))
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
news

,date,ngram,label
0,2017-01-02,"한국, 배터리, 보조금, 제외, 화학, 삼성, 날벼락, 한국, 배터리, 보조금, 제...",0
1,2017-01-02,"리더십, 시대, 분석, 산업, 혁명, 리더, 경영인, 누굴까, 리더십, 시대, 년,...",0
2,2017-01-02,"구본무, 회장, 창립, 올해, 시작, 구본무, 회장, 창립, 올해, 시작, 회장, ...",0
3,2017-01-02,"한국, 배터리, 설, 자리, 장착, 차량, 보조금, 제외, 국산, 배터리, 설, 자...",0
4,2017-01-02,"충전, 시간, 사용, 노트북, 시판, 충전, 시간, 사용, 노트북, 시판, 동아일보...",0
...,...,...,...
2878,2019-12-30,"화학, 놀이터, 화학, 캠프, 과학, 교육, 기부, 화학, 놀이터, 화학, 캠프, ...",1
2879,2019-12-30,"배터리, 시가총액, 돌파, 배터리, 시가총액, 돌파, 세계, 최대, 배터리, 제조사...",1
2880,2019-12-30,"전자, 폴란드, 브로츠와프, 배터리, 분리, 공장, 증설, 전자, 폴란드, 브로츠와...",1
2881,2019-12-30,"결산, 산업, 소송, 화재, 다사다난, 배터리, 업계, 결산, 산업, 소송, 화재,...",1


### NBC모델로 n-gram 감성지수 도출

In [ ]:
def my_tokenizer(x):
    return x.split(",")

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1), min_df = 10, tokenizer = my_tokenizer)),
                      ('clf', MultinomialNB(alpha=0.001))])
vect = text_clf.named_steps['vect']
clf = text_clf.named_steps['clf']

In [ ]:
accuracy = np.zeros(30)
posterior_list = []

for i in tqdm(range(30)):
    X_train, X_test, y_train, y_test = train_test_split(news['ngram'], news['label'], random_state = i, train_size = 0.9, shuffle=True)
    text_clf.fit(X_train, y_train)
    posterior_list.append(np.vstack([np.array(vect.get_feature_names()), np.exp(clf.feature_log_prob_)]))
    pred = text_clf.predict(X_test)
    accuracy[i] = np.sum(pred == y_test)

100%|██████████| 30/30 [06:21<00:00, 12.72s/it]


In [ ]:
clf.classes_

array([0, 1])

In [ ]:
pd.DataFrame(posterior_list)

In [ ]:
cv = CountVectorizer(ngram_range=(1,1), min_df = 10, tokenizer = my_tokenizer)

In [ ]:
X = cv.fit_transform(news['ngram'])

In [ ]:
ngram_list = cv.get_feature_names()
len(ngram_list)

13126

In [ ]:
polarity_scores = np.zeros((30, len(ngram_list)))

for i, itr in tqdm(enumerate(posterior_list)):
    for idx, n_gram in enumerate(itr[0]):
        tmp_n = ngram_list.index(n_gram)
        p_score = float(itr[2][idx])/float(itr[1][idx])
        polarity_scores[i][tmp_n] = p_score
        
polarity_scores.shape

30it [01:34,  3.16s/it]


(30, 13126)

In [ ]:
for i, v in enumerate(polarity_scores):
    for j, w in enumerate(v):
         if polarity_scores[i][j] == 0:
             polarity_scores[i][j] = None

In [ ]:
df_p_scores = pd.DataFrame(polarity_scores)
df_p_scores = df_p_scores.fillna(df_p_scores.mean())

In [ ]:
df_p_scores.T.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
dtype: int64

In [ ]:
avg_polarity_scores = list(df_p_scores.mean())

In [ ]:
ps = pd.DataFrame(avg_polarity_scores, index=ngram_list)
ps

,0
中,0.928444
新,0.966789
日,1.186878
美,0.859467
非,0.546174
...,...
포스코,0.525595
한국,0.981991
현대차,0.949639
화재,1.261666


In [ ]:
ps_filter = ps[ps[0].apply(lambda x: x > 1.2 or x < 0.8)]
ps_filter

,0
非,0.546174
韓,0.656229
가격,1.275462
가격;강세,4.842620
가격;경쟁력,1.585847
...,...
중국,1.384565
차세대,1.213854
포스코,0.525595
화재,1.261666


In [ ]:
ps_filter.reset_index(inplace=True)

In [ ]:
ps_filter.to_csv('/content/drive/MyDrive/DB_Finance/data/polarity_score_only_topic.csv', encoding = 'utf-8', index=False)